In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D,Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Reshape,TimeDistributed,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf

2024-06-13 14:50:07.136126: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Supongamos que 'data' es el DataFrame cargado con las columnas descritas
data=pd.read_csv('../../Data/Limpio/dataset_limpio_parte1.csv')
data1=pd.read_csv('../../Data/Limpio/dataset_limpio_parte2.csv')
data2=pd.read_csv('../../Data/Limpio/dataset_limpio_parte3.csv')
data_unida=pd.concat([data,data1,data2]) # Ajusta el nombre del archivo según corresponda
data_unida.head()

/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_1435/848381883.py:2: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('../../Data/Limpio/dataset_limpio_parte1.csv')
/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_1435/848381883.py:3: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data1=pd.read_csv('../../Data/Limpio/dataset_limpio_parte2.csv')
/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_1435/848381883.py:4: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data2=pd.read_csv('../../Data/Limpio/dataset_limpio_parte3.csv')


,cod_persona,mes,pais,sexo,edad,fecha1,xti_empleado,xti_nuevo_cliente,num_antiguedad,xti_rel,...,ind_prod16,ind_prod17,ind_prod18,ind_prod19,ind_prod20,ind_prod21,ind_prod22,ind_prod23,ind_prod24,ind_prod25
0,5514,2016-04-28,ES,H,44,2016-04-29,N,1.0,0,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
1,5541,2016-04-28,ES,H,60,2016-04-29,N,1.0,0,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
2,5631,2016-04-28,ES,V,53,2016-04-28,N,1.0,0,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
3,5656,2016-04-28,ES,H,47,2016-04-28,N,1.0,0,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
4,5738,2016-04-28,ES,V,48,2016-04-28,N,1.0,0,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0


In [3]:
mediana=data_unida[(data_unida['edad'] !='NA') & (data_unida['edad']!=' NA')]['edad'].astype(int).median()
print(mediana)
data_unida['edad']=data_unida['edad'].apply(lambda x: x if x not in ['NA',' NA'] else f'{mediana}')
data_unida['edad']=data_unida['edad'].astype(float)

39.0


In [4]:
mediana=data_unida[(data_unida['num_antiguedad'].str.strip() !='NA') & (data_unida['num_antiguedad']!=' NA')]['num_antiguedad'].astype(int).median()
print(mediana)
data_unida['num_antiguedad']=data_unida['num_antiguedad'].apply(lambda x: x if x not in ['NA','     NA'] else f'{mediana}')
data_unida['num_antiguedad']=data_unida['num_antiguedad'].astype(float)

50.0


In [5]:
data=data_unida[~data_unida['fecha1'].isna()]
data['fecha1'] = pd.to_datetime(data['fecha1'])
data['mes'] = pd.to_datetime(data['mes'])

/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_1435/883391504.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['fecha1'] = pd.to_datetime(data['fecha1'])
/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_1435/883391504.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['mes'] = pd.to_datetime(data['mes'])


In [6]:
columnas_cat=data.select_dtypes(include='O').columns

onehot=OneHotEncoder(sparse_output=False)
data_onehot=onehot.fit_transform(data[columnas_cat])
data_onehot=pd.DataFrame(data_onehot,columns=onehot.get_feature_names_out(columnas_cat))
df_onehot = data_onehot.reset_index(drop=True)
data = data.reset_index(drop=True)

data=pd.concat([data.drop(columnas_cat,axis=1),df_onehot],axis=1)
data.head()

,cod_persona,mes,edad,fecha1,xti_nuevo_cliente,num_antiguedad,xti_rel,tip_dom,cod_provincia,xti_actividad_cliente,...,des_canal_KHO,des_canal_KHP,des_canal_KHQ,des_canal_RED,xti_extra_N,xti_extra_S,id_segmento_01 - TOP,id_segmento_02 - PARTICULARES,id_segmento_03 - UNIVERSITARIO,id_segmento_04-NO TIENE
0,5514,2016-04-28,44.0,2016-04-29,1.0,0.0,1.0,1.0,8.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,5541,2016-04-28,60.0,2016-04-29,1.0,0.0,1.0,1.0,41.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,5631,2016-04-28,53.0,2016-04-28,1.0,0.0,1.0,1.0,15.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,5656,2016-04-28,47.0,2016-04-28,1.0,0.0,1.0,1.0,46.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,5738,2016-04-28,48.0,2016-04-28,1.0,0.0,1.0,1.0,28.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [7]:
data = data.sort_values(by=['cod_persona', 'mes'])
cods_persona=data['cod_persona']
data.drop('fecha1',axis=1,inplace=True)

#Prueba sin secuencias
feature_cols = [col for col in data.columns if not col =='ind_prod8' and col not in ['cod_persona', 'mes']]
target_cols = [col for col in data.columns if col =='ind_prod8']
def crear_modelo_mixto(input_shape, output_shape,dense_units=[], dropout_rate=0.5):
    model = Sequential()
    
    for idx,i in enumerate(dense_units):
        if idx==0:
            model.add(Dense(units=i,activation='relu',input_dim=input_shape))
            model.add(Dropout(dropout_rate))
        else:
            model.add(Dense(units=i,activation='relu'))
            model.add(Dropout(dropout_rate))

    # Capa de salida
    model.add(Dense(output_shape, activation='sigmoid'))
    
    # Compilar el modelo
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy','recall'])
    
    return model
X=data[feature_cols]
y=data[target_cols]
cutoff = int(len(X) * 0.8)
# Dividir los datos en entrenamiento y prueba manteniendo la temporalidad
X_train, X_test = X[:cutoff], X[cutoff:]
y_train, y_test = y[:cutoff], y[cutoff:]

X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)

y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)

print(X_train_tensor)
model=crear_modelo_mixto(X_train_tensor.shape[1],y_train_tensor.shape[1],dense_units=[120,80,60,40],dropout_rate=.5)
model.summary()
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=1),  # Parada temprana si la pérdida en validación deja de disminuir
    ModelCheckpoint('modelo_epoch_{epoch:02d}.keras', save_freq='epoch', verbose=1)  # Guardar el modelo en cada época
]
history=model.fit(X_train_tensor,y_train_tensor,epochs=10,batch_size=16,validation_data=(X_test_tensor,y_test_tensor),callbacks=callbacks)

In [8]:
import numpy as np

def create_sequences(df, cod_persona_col, feature_cols, target_cols, sequence_length):
    sequences = []
    targets = []
    targets_anteriores = []

    personas = df[cod_persona_col].unique()

    # Convertir el DataFrame a un numpy array para operaciones más rápidas
    df_values = df[[cod_persona_col] + feature_cols + target_cols].values
    cod_persona_index = 0  # El índice de cod_persona_col en df_values

    for persona in personas:
        # Filtrar las filas correspondientes a la persona actual
        persona_mask = df_values[:, cod_persona_index] == persona
        persona_df = df_values[persona_mask]
        
        # Si la longitud del DataFrame de la persona es menor que sequence_length, se omite
        if len(persona_df) <= sequence_length:
            continue
        
        for i in range(len(persona_df) - sequence_length):
            sequence = persona_df[i:i+sequence_length, 1:1+len(feature_cols)]
            target = persona_df[i+sequence_length, 1+len(feature_cols):1+len(feature_cols)+len(target_cols)]
            target_anterior = persona_df[i+sequence_length-1, 1+len(feature_cols):1+len(feature_cols)+len(target_cols)]
            sequences.append(sequence)
            targets.append(target)
            targets_anteriores.append(target_anterior)
    
    return np.array(sequences), np.array(targets), np.array(targets_anteriores)

# Definir las columnas de características y de objetivos
feature_cols = [col for col in data.columns if col.find('ind_prod')==-1 and col not in ['cod_persona', 'mes']]
target_cols = [col for col in data.columns if col.find('ind_prod')!=-1 and col.find('ind_prod2')==-1]

# Longitud de la secuencia
sequence_length = 4  # Por ejemplo, secuencias de 5 días

# Crear las secuencias
X, y, y_prev = create_sequences(data, 'cod_persona', feature_cols, target_cols, sequence_length)


In [9]:
cutoff = int(len(X) * 0.8)
# Dividir los datos en entrenamiento y prueba manteniendo la temporalidad
X_train, X_test = X[:cutoff], X[cutoff:]
y_train, y_test = y[:cutoff], y[cutoff:]
y_prev_train, y_prev_test = y_prev[:cutoff], y_prev[cutoff:]

X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)

y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)

y_prev_train_tensor = tf.convert_to_tensor(y_prev_train, dtype=tf.float32)
y_prev_test_tensor = tf.convert_to_tensor(y_prev_test, dtype=tf.float32)

In [10]:
def crear_modelo_mixto(input_shape, output_shape,  lstm_units=204,dense_units=[], dropout_rate=0.5):
    model = Sequential()
    model.add(LSTM(units=lstm_units, return_sequences=True, dropout=dropout_rate, input_shape=input_shape))
    model.add(LSTM(units=lstm_units, return_sequences=False, dropout=dropout_rate, input_shape=input_shape))
    
    # Añadir capas Conv2D y MaxPooling2D
    for i in dense_units:
        model.add(Dense(units=i,activation='relu'))
        model.add(Dropout(dropout_rate))

    # Capa de salida
    model.add(Dense(output_shape, activation='sigmoid'))
    
    # Compilar el modelo
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy','recall','categorical_accuracy'])
    
    return model

In [11]:
X_train_tensor.shape

TensorShape([359200, 4, 204])

In [12]:
model=crear_modelo_mixto(X_train_tensor.shape[-2:],y_test_tensor.shape[1],lstm_units=204,dense_units=[120,120,120,120,100,80,60,40],dropout_rate=.5)
model.summary()

/Users/mariolamas/anaconda3/envs/entorno_prueba/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 4, 204)         │       333,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 204)            │       333,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 120)            │        24,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 120)            │        14,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 120)            │        14,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 120)            │        14,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 100)            │        12,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 80)             │         8,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 80)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 60)             │         4,860 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 60)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 40)             │         2,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 40)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 18)             │           738 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 763,866 (2.91 MB)

 Trainable params: 763,866 (2.91 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=1),  # Parada temprana si la pérdida en validación deja de disminuir
    ModelCheckpoint('modelo_epoch_{epoch:02d}.keras', save_freq='epoch', verbose=1)  # Guardar el modelo en cada época
]

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=1),  # Parada temprana si la pérdida en validación deja de disminuir
    ModelCheckpoint('modelo_epoch_{epoch:02d}.keras', save_freq='epoch', verbose=1)  # Guardar el modelo en cada época
]
history=model.fit(X_train_tensor,y_train_tensor,epochs=10,batch_size=16,validation_data=(X_test_tensor,y_test_tensor),callbacks=callbacks)

In [15]:
print(X_train_tensor[1][-1])
#Prueba de prediccion sobre 1 ejemplo
y_pred=model.predict(tf.expand_dims(X_train_tensor[1],axis=0))
y_pred

tf.Tensor(
[33.        1.        3.        1.        1.        8.        1.
 11.562021  5.963083  0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        1.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        1.        0.        0.        0.        0.        1.
  1.        0.        0.        0.        0.        0.        1.
  1.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.

array([[1.4588236e-05, 6.7047918e-01, 2.6254560e-05, 8.9422032e-02,
        1.0117410e-02, 7.0739025e-03, 7.7753589e-02, 3.3857290e-02,
        6.9553083e-05, 1.4256542e-04, 3.4024056e-02, 8.4189132e-02,
        5.2291490e-03, 6.5720751e-04, 1.8762107e-03, 5.5288529e-04,
        6.3034669e-02, 3.2399591e-02]], dtype=float32)

In [16]:
y_train_tensor[1]

<tf.Tensor: shape=(18,), dtype=float32, numpy=
array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.], dtype=float32)>

In [17]:
loss, accuracy = model.evaluate(X_test_tensor, y_test_tensor)
print(f"Loss: {loss}, Accuracy: {accuracy}")

# Definir la ruta donde se guardará el modelo
model_path = './mi_modelo.h5'

# Guardar el modelo
model.save(model_path)
print(f'Modelo guardado en: {model_path}')

2807/2807 ━━━━━━━━━━━━━━━━━━━━ 34s 12ms/step - accuracy: 0.5582 - categorical_accuracy: 0.5582 - loss: 0.2317 - recall: 0.3203


ValueError: too many values to unpack (expected 2)

In [ ]:
# Cargar el modelo
modelo_cargado = load_model('./mi_modelo.h5')

# Verificar que el modelo ha sido cargado correctamente
modelo_cargado.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 4, 64)          │        68,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 81)             │         5,265 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 81)             │         6,642 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 81)             │         6,642 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 81)             │         6,642 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 81)             │         6,642 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 25)             │         2,050 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 135,773 (530.37 KB)

 Trainable params: 135,771 (530.36 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [ ]:
import pickle

# Definir la ruta donde se guardará el historial
history_path = './history.pkl'

# Guardar el historial
with open(history_path, 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
# Cargar el historial
with open(history_path, 'rb') as file_pi:
    history = pickle.load(file_pi)

print('Historial cargado correctamente')

In [328]:
#Prueba de prediccion con map@k
import numpy as np

def apk(actual, predicted, k):
    if len(predicted)>k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)


def mapk(actual, predicted, k):    
  return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [329]:
#prueba implementacion map
def preparacion(predictions,inds,y_trues,y_prevs):
    for p,t,prev in zip(predictions,y_trues,y_prevs):
        assert len(inds) == len(p), "Las listas inds y predictions deben tener la misma longitud"
        assert len(inds) == len(t)
        assert len(inds) == len(prev)

        real_pro=[]
        pred_pro=[]

        # Crear una lista de tuplas (prediccion, etiqueta)
        predicciones_con_etiquetas = list(zip(p, inds,prev))
        threshold=.01
        predicciones_filtradas = [(tupla[0],tupla[1]) for tupla in predicciones_con_etiquetas if (tupla[0] > threshold) and (tupla[2]!=1)]
        # Ordenar las tuplas por el valor de la prediccion en orden descendente
        predicciones_ordenadas = [et for punt,et in sorted(predicciones_filtradas, key=lambda x: x[0], reverse=True)]

        t_p=[i for y_t,i in zip(t,inds) if y_t==1]

        pred_pro.append(predicciones_ordenadas)
        real_pro.append(t_p)
    return pred_pro,real_pro



In [ ]:
predicciones=[0 for idx, m in enumerate(models) if idx==valor else m.predict(datos)]

In [330]:
y_train_tensor.shape

TensorShape([359200, 18])

In [334]:
inds=target_cols
predictions=[model.predict(tf.expand_dims(X_train_tensor[2000],axis=0)).tolist()[0]]
y_true=[y_train_tensor[2000].numpy().tolist()]
y_prevs=[y_prev_train_tensor[2000].numpy().tolist()]
predicciones,reales=preparacion(predictions,inds,y_true,y_prevs)
print(predicciones)
print(reales)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[['ind_prod13', 'ind_prod8', 'ind_prod5', 'ind_prod18', 'ind_prod9', 'ind_prod12', 'ind_prod19', 'ind_prod6', 'ind_prod14', 'ind_prod7']]
[['ind_prod3']]


In [333]:
np.mean([mapk(reales,predicciones,i) for i in range(2,10)])

0.0